In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate())
    
    x_prev = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1  = add!(model, constvar(:add_1, 1.0))
    x      = add!(model, randomvar(:x))
    
    x_prev_add = add!(model, make_node(+, x_prev, add_1, x))
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    y     = add!(model, datavar(:y, Float64))
    
    add_x_and_noise = add!(model, make_node(+, x, noise, y))
    
    activate!(model)
    
    return x_prev, x, y
end

function kalman(data)
    N = length(data)
    
    x_prev, x, y = kalman_filter_graph()

    link(x_prev, getmarginal(x))
    
    update!(x_prev, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [3]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [4]:
@time kalman(data); # Initial compilation

  2.893593 seconds (8.52 M allocations: 447.284 MiB, 3.68% gc time)


In [5]:
@time kalman(data); # Subsequent runs

  0.000914 seconds (16.12 k allocations: 802.062 KiB)


In [6]:
@btime kalman($data) # Performance benchmark

  630.588 μs (16123 allocations: 802.06 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=0.16010367468364783, σ=14.002800840280099)
 Normal{Float64}(μ=-7.0087700521219825, σ=9.950371902099892)
 Normal{Float64}(μ=3.4672168252042246, σ=8.137884587711596)
 Normal{Float64}(μ=7.294302504504734, σ=7.053456158585984)
 Normal{Float64}(μ=4.216026949682214, σ=6.311944030978033)
 Normal{Float64}(μ=8.398396804758855, σ=5.763904177042351)
 Normal{Float64}(μ=11.514647805357082, σ=5.337605126836238)
 Normal{Float64}(μ=8.91920860641781, σ=4.993761694389224)
 Normal{Float64}(μ=9.145433030833095, σ=4.708816093480111)
 Normal{Float64}(μ=8.336061457798774, σ=4.467670516087703)
 Normal{Float64}(μ=10.817223153949357, σ=4.2601432284230505)
 Normal{Float64}(μ=12.827508015918768, σ=4.079085082240021)
 Normal{Float64}(μ=13.990436471420942, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.588819363488, σ=0.582706653680032)
 Normal{Float64}(μ=589.5929099136839, σ=0.5822126418123303)
 Normal{Float64}(μ=590.6135088533742, σ=0.5817198842703781)
 Nor